In [1]:
from lib.sentence2vec import Sentence2Vec
import pandas as pd

In [2]:
data = pd.read_csv('../feature/data_cutted.csv')
model = Sentence2Vec('../feature/content_small.model')

sen2vec = pd.DataFrame(data['content'].apply(lambda x:model.get_vector(str(x))).tolist())

data_df = pd.concat([data,sen2vec],axis=1)
data_df.drop(['content'],axis=1,inplace=True)

#data_df.to_csv('../feature/data_vec.csv',index=False)
#data_df = pd.read_csv('../feature/data_vec.csv')

train = data_df.loc[data_df['source']=='train']
test = data_df.loc[data_df['source']=='test']

train.drop(['source'],axis=1,inplace=True)
test.drop(['source'],axis=1,inplace=True)

del data_df

/Users/zhoujingjing/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [3]:
train.head()

,id,0,1,2,3,4,5,6,7,8,...,290,291,292,293,294,295,296,297,298,299
0,yhM4ivIm,-0.000489,0.006424,-0.000634,0.000838,0.011319,0.000702,-0.003625,-0.000884,-0.000510,...,0.002738,-0.002048,0.000028,-0.003345,-0.001122,0.001509,0.001144,-0.009643,0.001227,0.001722
1,OeJidjGX,-0.000468,0.006323,-0.002555,0.001104,0.014201,-0.001923,-0.009285,-0.005986,0.009980,...,0.008317,-0.004408,-0.005493,0.003575,-0.006894,-0.008139,-0.002329,-0.006802,0.005028,0.000220
2,4wWp3dGK,-0.001736,0.002232,0.000045,-0.004270,0.004914,0.004732,0.000825,0.000135,0.001287,...,-0.003094,-0.010099,-0.005695,-0.012348,-0.004554,0.001013,-0.000672,-0.004208,0.000265,0.001462
3,T0senQto,-0.004385,0.006698,-0.005495,-0.004874,0.019540,-0.002231,0.007779,-0.005792,-0.001375,...,0.012416,-0.010862,-0.007198,-0.008945,-0.009026,-0.018687,0.010948,-0.022148,0.014432,0.002735
4,1jAboq7C,0.000545,0.001087,0.002131,-0.000466,0.003713,-0.000793,-0.000485,0.001973,-0.002907,...,0.001476,-0.002533,-0.001731,-0.002808,-0.000747,-0.000633,0.000697,0.002841,0.004798,-0.000442


In [4]:
X = train.drop(['id'],axis=1)

In [5]:
y = train['id']

In [6]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=1)
neigh.fit(X, y) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform')

In [7]:
feature = test.drop(['id'],axis=1)
result = neigh.predict(feature)

sub = pd.DataFrame({'0':test['id'],'1':result})
sub.columns=['test_id','result']

In [8]:
# compute similar
sub.head()

,test_id,result
99544,ahpq2Zzn,OBIWG4sg
99545,p2Hkot4E,ZjlDhAMS
99546,6pEJaYSB,xqeJwCQb
99547,fEClIHUb,DtWPQnv4
99548,QORyZCX6,viqACuS2


In [9]:
def get_distance(x,y):
    return model.similarity(data[data.id==x]['content'].to_string(),data[data.id==y]['content'].to_string())

sub['distance']=sub.apply(lambda row: get_distance(row['test_id'], row['result']), axis=1)

In [10]:
sub.head()

,test_id,result,distance
99544,ahpq2Zzn,OBIWG4sg,0.802831
99545,p2Hkot4E,ZjlDhAMS,0.724095
99546,6pEJaYSB,xqeJwCQb,0.842839
99547,fEClIHUb,DtWPQnv4,0.730220
99548,QORyZCX6,viqACuS2,0.701679


In [11]:
sub.describe()

,distance
count,1000.000000
mean,0.817399
std,0.132230
min,0.210324
25%,0.731937
50%,0.828520
75%,0.929022
max,1.000000


In [12]:
def threshold(result,distance):
    if distance > sub['distance'].quantile(.5):
        return result
    else:
        return 

In [13]:
sub['result'] = sub.apply(lambda row: threshold(row['result'],row['distance']),axis=1)

In [14]:
sub.drop('distance',inplace=True,axis=1)

In [15]:
sub.replace({'﻿mhY5opF':'?mhY5opF4'},inplace=True)

In [16]:
sub.to_csv('../output/sen2vec300small_threshold50.csv',index=False)